# Aplicația practică la disciplina Prelucrarea imaginilor în realitatea virtuală:
# 1.Definirea problemei:
Dezvoltarea unei rețele neuronale convoluționale; să antrenez un model pentru a recunoaște imaginile din dataset-ul CIFAR10 pe 10 categorii.

In [1]:
## configurarea stackului: 
## un distributiv python sau anaconda; am folosit VSCode cu python 3.9 și extesia Jupyter
#update python la versiunea 3.11
## install pytorch torchvision cpuonly -c pytorch (dacă nu avem GPU cu nucleu CUDA )
## pip install pytorch torchvision pytorch-cuda -c pytorch -c nvidia (GPU nvidia cu nuclee CUDA)
## install streamlit - pentru implementarea moselului pe platforma și a demonstra funcționalitatea 

2. Colectarea datelor: import biblioteca PyTorch, modulul pentru rețea neuronală, biblioteca torchvision, NumPy și modulul pyplot din biblioteca matplotlib

In [3]:
import torch
print(torch.cuda.is_available())
import torch.nn as nn
import torchvision
import math
from matplotlib import pyplot as plt
import numpy as np
import torch.nn.functional as F
#device=torch.device("cpu:0") #daca rulezi pe cpu
device=torch.device("cuda:0")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu") ## nu nu lucrează oe toate dispozitivile

True


3. Pregătirea datelor: definirea unei transformări a datelor care convertește imaginile de intrare în tensori PyTorch și le normalizează, setarea dimensiunii lotului și crearea încărcătoarelor de date pentru setul de date CIFAR10.

In [4]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4 # atent la training pe procesor

trainset = torchvision.datasets.CIFAR10(root='/tmp/cifar10', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='/tmp/cifar10', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck') #de făcut un dataset pentru încercări

Files already downloaded and verified
Files already downloaded and verified


4. Algoritmul: definirea unei clase ConvModel care extinde clasa PyTorch nn.Module. Această clasă definește arhitectura modelului CNN, care constă din două straturi convoluționale urmate de două straturi complet conectate. Metoda forward implementează trecerea înainte a modelului.

In [5]:
class ConvModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels= 32,
            kernel_size=3, 
            padding=1)
        
        self.pool = nn.MaxPool2d(
            kernel_size=2, 
            stride=2)
        
        self.conv2 = nn.Conv2d(
            in_channels=32, 
            out_channels=64, 
            kernel_size=3,
            padding=1)
        
        self.fc1 = nn.Linear(
            in_features=64 * 8 * 8, 
            out_features=512)
        
        self.fc2 = nn.Linear(in_features=512, 
                             out_features=10)
        
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)# de verificat dimensiunea la flatten; mai optim ar fi
        #x = x.view(x.size(0), -1)

        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model = ConvModel().to(device)
model

ConvModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (relu): ReLU()
)

4.1 Optimizarea modelului: definirea funcției de pierdere (cross-entropy) și a optimizatorului (coborâre stocastică a gradientului).

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

5. Formarea modelului: formarea modelului timp de două epoci utilizând datele de formare. În fiecare epocă, iterez peste loturile din datele de instruire, calculează trecerile înainte și înapoi și actualizează ponderile modelului utilizând optimizatorul.

In [ ]:

model.train()
for epoch in range(10):  # parcurgerea în buclă a setului de date de n ori

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs=inputs.to(device)
        labels=labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            #stats: pe gtx 1660ti a fost 23min
            #rtx 3060ti 8 min a durat 


[1,  2000] loss: 1.950
[1,  4000] loss: 1.563
[1,  6000] loss: 1.392
[1,  8000] loss: 1.319
[1, 10000] loss: 1.214
[1, 12000] loss: 1.165
[2,  2000] loss: 1.057
[2,  4000] loss: 0.991
[2,  6000] loss: 0.989
[2,  8000] loss: 0.959
[2, 10000] loss: 0.921
[2, 12000] loss: 0.908
[3,  2000] loss: 0.771
[3,  4000] loss: 0.790
[3,  6000] loss: 0.761
[3,  8000] loss: 0.750
[3, 10000] loss: 0.760
[3, 12000] loss: 0.731
[4,  2000] loss: 0.569
[4,  4000] loss: 0.579
[4,  6000] loss: 0.591
[4,  8000] loss: 0.592
[4, 10000] loss: 0.602
[4, 12000] loss: 0.587
[5,  2000] loss: 0.379
[5,  4000] loss: 0.398
[5,  6000] loss: 0.406
[5,  8000] loss: 0.433
[5, 10000] loss: 0.447
[5, 12000] loss: 0.448
[6,  2000] loss: 0.220
[6,  4000] loss: 0.249
[6,  6000] loss: 0.261
[6,  8000] loss: 0.273
[6, 10000] loss: 0.292
[6, 12000] loss: 0.317
[7,  2000] loss: 0.126
[7,  4000] loss: 0.137
[7,  6000] loss: 0.156
[7,  8000] loss: 0.176
[7, 10000] loss: 0.188
[7, 12000] loss: 0.204
[8,  2000] loss: 0.083
[8,  4000] 

6. Evaluare: evaluarea performanței modelului antrenat pe datele de testare prin calcularea preciziei modelului. Setați modelul în modul de evaluare pentru a dezactiva abandonul și normalizarea loturilor, apoi iterați pe datele de testare pentru a calcula precizia.

In [ ]:
#test precizie - obiectiv mai mult de 70 la sută
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images=images.to(device)
        labels=labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Precizia desusă în urma încercărilor făcute în baza a 10000 imagini: %f %%' % (
    100 * correct / total))
#rev 2 - 73,4%

Precizia desusă în urma încercărilor făcute în baza a 10000 imagini: 72.640000 %


Această rețea neuronală convoluțională (CNN) este un model de învățare profundă care este antrenat pe setul de date CIFAR-10 pentru clasificarea imaginilor. Setul de date CIFAR-10 constă din 60 000 de imagini color 32x32 în 10 clase, cu 6 000 de imagini pe clasă. Arhitectura modelului constă din două straturi convoluționale și două straturi complet conectate. Primul strat convoluțional are 32 de filtre de dimensiune 3x3, iar al doilea strat convoluțional are 64 de filtre de dimensiune 3x3. Ambele straturi convoluționale sunt urmate de un strat max pooling care reduce dimensiunile spațiale ale hărților de caracteristici. Ieșirea celui de-al doilea strat max pooling este aplatizată și introdusă în două straturi complet conectate, primul având 512 neuroni, iar al doilea având 10 neuroni (câte unul pentru fiecare clasă din setul de date). Funcția de activare ReLU este utilizată după fiecare strat convoluțional și complet conectat, cu excepția ultimului, care utilizează o funcție de activare softmax pentru a emite probabilitățile ca imaginea de intrare să aparțină fiecărei clase. Modelul este antrenat utilizând metoda stochastic gradient descent (SGD) cu o rată de învățare de 0,001 și un momentum de 0,9, iar funcția de pierdere a entropiei încrucișate este utilizată ca obiectiv de optimizare.


7. implementare: am folosit biblioteca Streamlit, iar acum pot crea o interfață care să permită utilizatorilor să interacționeze cu modelul dvs. 

In [9]:
torch.save(model, 'cifar10_model.pth')